<a href="https://colab.research.google.com/github/qubit19/AlgosInCuda/blob/main/AlgosComparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update
!apt install -y build-essential
!apt install -y cuda-toolkit-11-8


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,607 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,701 kB]
Get:13 https://r2u.stat.illinois.edu/u

In [ ]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.4 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1-cp311-cp311-linux_x86_64.whl size=660426 sha256=37a3456a69d32a37ea638c0ba0275ce70324a8af5af0d9341c30877256621791
  Stored in directory: /root/.cache/pip/wheels/77/7e/6c/d2d1451ea6424cdc3d67b36c16fa7111eafdf2034bc3405666
Successfully built pycuda


In [ ]:


import pycuda.driver as drv
import pycuda.autoinit # Initializes CUDA context
import pycuda.compiler as compiler
import numpy as np
import time
import collections
import heapq

# Check PyCUDA initialization
try:
    print(f"PyCUDA initialized successfully on device: {drv.Device(0).name()}")
    !nvidia-smi # Display GPU information again
except Exception as e:
    print("Error initializing PyCUDA:", e)
    print("Please ensure CUDA drivers and toolkit are compatible and a GPU runtime is selected.")
    # import sys
    # sys.exit()

PyCUDA initialized successfully on device: Tesla T4
Sat Apr 26 21:00:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P0             26W /   70W |     206MiB /  15360MiB |      1%      Default |
|                                         |                        |                  N/

In [ ]:
# @title 1. Graph Representation (CSR Format) and Sample Graph



def create_sample_graph_csr(num_nodes, num_edges, directed=False, seed=42):
    """Creates a random sparse graph and returns its CSR representation."""
    print(f"Creating a sample graph: {num_nodes} nodes, ~{num_edges} edges...")
    np.random.seed(seed)
    max_edges = num_nodes * (num_nodes - 1)
    if not directed: max_edges //= 2
    num_edges = min(num_edges, max_edges)
    sources = np.random.randint(0, num_nodes, num_edges * 2)
    targets = np.random.randint(0, num_nodes, num_edges * 2)
    mask = sources != targets
    sources = sources[mask][:num_edges]
    targets = targets[mask][:num_edges]
    if not directed:
        sources_full = np.concatenate([sources, targets])
        targets_full = np.concatenate([targets, sources])
    else:
        sources_full = sources
        targets_full = targets
    weights = np.random.randint(1, 11, size=len(sources_full)).astype(np.float32)
    sorted_indices = np.argsort(sources_full)
    sources_sorted = sources_full[sorted_indices]
    targets_sorted = targets_full[sorted_indices]
    weights_sorted = weights[sorted_indices]
    indptr = np.zeros(num_nodes + 1, dtype=np.int32)
    indices = np.zeros(len(targets_sorted), dtype=np.int32)
    data = np.zeros(len(weights_sorted), dtype=np.float32)
    current_edge_idx = 0
    last_source = -1
    for i in range(len(sources_sorted)):
        source = sources_sorted[i]
        target = targets_sorted[i]
        weight = weights_sorted[i]
        while last_source < source:
            last_source += 1
            indptr[last_source] = current_edge_idx
        indices[current_edge_idx] = target
        data[current_edge_idx] = weight
        current_edge_idx += 1
    while last_source < num_nodes:
        last_source += 1
        indptr[last_source] = current_edge_idx
    indptr[num_nodes] = current_edge_idx
    indices = indices[:current_edge_idx]
    data = data[:current_edge_idx]
    print(f"Actual number of edges stored (considering directionality): {current_edge_idx}")
    print("Graph representation (CSR):")
    print(f"  indptr (shape {indptr.shape}, {indptr.dtype})")
    print(f"  indices (shape {indices.shape}, {indices.dtype})")
    print(f"  data (shape {data.shape}, {data.dtype})")
    return indptr, indices, data, num_nodes

# --- Create a SMALL graph for demonstration ---
NUM_NODES = 1024 * 4 # Increase size slightly for better comparison
AVG_DEGREE = 16
NUM_EDGES = NUM_NODES * AVG_DEGREE

csr_indptr_cpu, csr_indices_cpu, csr_weights_cpu, N = create_sample_graph_csr(NUM_NODES, NUM_EDGES, directed=False)

# --- Helper function for PyCUDA kernel launch config ---
def get_kernel_config_pycuda(n_items, threads_per_block=256):
    """ Calculates grid and block dimensions for PyCUDA. """
    blocks_per_grid = (n_items + threads_per_block - 1) // threads_per_block
    return (int(blocks_per_grid), 1, 1), (int(threads_per_block), 1, 1) # PyCUDA expects tuples

Creating a sample graph: 4096 nodes, ~65536 edges...
Actual number of edges stored (considering directionality): 131072
Graph representation (CSR):
  indptr (shape (4097,), int32)
  indices (shape (131072,), int32)
  data (shape (131072,), float32)


In [ ]:
# @title 2. PyCUDA Kernels and Host Code

# --- 2.1 Breadth-First Search (BFS) with PyCUDA ---

bfs_kernel_code_pycuda = r'''
// Same kernel code as before
extern "C" __global__
void bfs_kernel(const int* indptr, const int* indices, int* dist, int* frontier_in, int* frontier_out, int* frontier_size_out, int current_level, int num_nodes) {

    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    // Read current frontier size ONCE from global memory (atomically updated by host/other blocks)
    // Use volatile to prevent compiler optimizing reads, though atomic access handles memory visibility.
    // int current_frontier_size = frontier_size_out[0]; // This was incorrect logic in CuPy version too for kernel boundary check!
    // The check needs to be against the size *before* the kernel started adding to frontier_out.
    // Let's assume the grid size was calculated correctly based on frontier_size_in
    // The size passed in frontier_size_out[0] is the *input* size for this kernel launch.

    if (tid < frontier_size_out[0]) { // Check if thread ID is within the input frontier bounds
         int u = frontier_in[tid]; // Get node 'u' from the current frontier

         if (u < 0 || u >= num_nodes) return; // Boundary check for safety

         int start_edge = indptr[u];
         int end_edge = indptr[u + 1];

         for (int i = start_edge; i < end_edge; ++i) {
             int v = indices[i]; // Neighbor node 'v'

             if (v < 0 || v >= num_nodes) continue; // Boundary check

             int old_dist = atomicCAS(&dist[v], -1, current_level + 1);

             if (old_dist == -1) {
                 // Add 'v' to the *next* frontier using atomicAdd to get a unique index
                 int index = atomicAdd(&frontier_size_out[1], 1); // Index for next frontier size
                 // *** Potential Issue: If next frontier exceeds allocation size, this writes out of bounds! ***
                 // A robust implementation needs size checks or dynamic resizing (complex).
                 // Assuming frontier arrays are allocated large enough (num_nodes).
                 frontier_out[index] = v; // Add 'v' to the next frontier array
             }
         }
    }
}
'''

# Compile the kernel
bfs_module = compiler.SourceModule(bfs_kernel_code_pycuda)
bfs_kernel_pycuda = bfs_module.get_function("bfs_kernel")

def run_bfs_gpu_timed_pycuda(indptr_cpu, indices_cpu, num_nodes, start_node=0):
    """ Executes BFS on the GPU using PyCUDA with timing """
    print(f"\nRunning BFS from node {start_node} on GPU (PyCUDA)...")

    # Allocate GPU memory
    d_indptr = drv.mem_alloc(indptr_cpu.nbytes)
    d_indices = drv.mem_alloc(indices_cpu.nbytes)
    d_dist = drv.mem_alloc(num_nodes * np.int32().itemsize)
    d_frontier1 = drv.mem_alloc(num_nodes * np.int32().itemsize)
    d_frontier2 = drv.mem_alloc(num_nodes * np.int32().itemsize)
    d_frontier_size = drv.mem_alloc(2 * np.int32().itemsize) # [current_size, next_size]

    # Transfer initial data H->D
    drv.memcpy_htod(d_indptr, indptr_cpu)
    drv.memcpy_htod(d_indices, indices_cpu)

    # Initialize dist array (-1)
    dist_init_cpu = np.full(num_nodes, -1, dtype=np.int32)
    drv.memcpy_htod(d_dist, dist_init_cpu)

    # Initialize frontier size array ([0, 0])
    frontier_size_init_cpu = np.zeros(2, dtype=np.int32)
    drv.memcpy_htod(d_frontier_size, frontier_size_init_cpu)

    # Setup initial state on CPU first, then transfer
    dist_init_cpu[start_node] = 0
    drv.memcpy_htod(d_dist, dist_init_cpu) # Transfer updated dist

    frontier1_init_cpu = np.zeros(num_nodes, dtype=np.int32) # Only need first element
    frontier1_init_cpu[0] = start_node
    drv.memcpy_htod(d_frontier1, frontier1_init_cpu) # Transfer updated frontier1

    frontier_size_init_cpu[0] = 1 # Set current size to 1
    drv.memcpy_htod(d_frontier_size, frontier_size_init_cpu) # Transfer updated size

    current_level = 0
    threads_per_block = 256

    frontier_in_gpu = d_frontier1
    frontier_out_gpu = d_frontier2

    # Timing events
    start_event = drv.Event()
    end_event = drv.Event()
    total_gpu_time = 0.0

    # Temporary CPU array for frontier size
    frontier_size_h = np.zeros(2, dtype=np.int32)
    frontier_size_h_reset_next = np.zeros(2, dtype=np.int32) # Used to reset next_size to 0

    # --- Main BFS loop (level-synchronous) ---
    start_event.record() # Record event before the loop
    while True:
        # Read current frontier size D->H
        drv.memcpy_dtoh(frontier_size_h, d_frontier_size)
        current_frontier_size = int(frontier_size_h[0])

        if current_frontier_size == 0:
            break # No more nodes to visit

        # Reset next frontier size counter ([current, 0]) H->D
        frontier_size_h_reset_next[0] = current_frontier_size # Keep current size
        frontier_size_h_reset_next[1] = 0                     # Reset next size
        drv.memcpy_htod(d_frontier_size, frontier_size_h_reset_next)

        # Configure and launch kernel
        grid_dim, block_dim = get_kernel_config_pycuda(current_frontier_size, threads_per_block)

        # Prepare args: Pointers and explicitly typed scalars
        args = [d_indptr, d_indices, d_dist, frontier_in_gpu, frontier_out_gpu,
                d_frontier_size, np.int32(current_level), np.int32(num_nodes)]
        bfs_kernel_pycuda(*args, block=block_dim, grid=grid_dim)

        # Read the size of the *next* frontier D->H (needed to set current size for next iter)
        drv.memcpy_dtoh(frontier_size_h, d_frontier_size) # This syncs kernel implicitly
        next_frontier_size = int(frontier_size_h[1])

        # Update current frontier size on GPU H->D ([next, next])
        frontier_size_h[0] = next_frontier_size # Set current = next for next loop
        # frontier_size_h[1] is already next_frontier_size from the memcpy_dtoh above
        drv.memcpy_htod(d_frontier_size, frontier_size_h)

        # Swap frontier pointers (no data movement)
        frontier_in_gpu, frontier_out_gpu = frontier_out_gpu, frontier_in_gpu

        current_level += 1

    end_event.record() # Record event after the loop
    # --- End BFS Loop ---

    # Wait for all GPU activity to complete
    end_event.synchronize()
    gpu_time = start_event.time_till(end_event) / 1000.0 # Time in seconds

    # Retrieve results D->H
    dist_result_cpu = np.empty_like(dist_init_cpu)
    drv.memcpy_dtoh(dist_result_cpu, d_dist)

    # Free GPU memory (important!)
    d_indptr.free()
    d_indices.free()
    d_dist.free()
    d_frontier1.free()
    d_frontier2.free()
    d_frontier_size.free()

    print(f"GPU BFS (PyCUDA) finished in {gpu_time:.4f} seconds.")
    unvisited_count = np.sum(dist_result_cpu == -1)
    print(f"Number of nodes reachable from start node {start_node}: {num_nodes - unvisited_count}")

    return dist_result_cpu, gpu_time


# --- 2.2 Dijkstra's Algorithm with PyCUDA ---

dijkstra_kernel_code_pycuda = r'''
#include <float.h> // For FLT_MAX

// Need bool definition for kernel if not included elsewhere
// typedef unsigned char bool;
// #define true 1
// #define false 0
// PyCUDA seems to handle bool okay usually.

// atomicCAS for float simulation
static __device__ float atomicMinFloat(float* addr, float value) {
    float old;
    unsigned int* addr_as_int = (unsigned int*)addr;
    unsigned int old_int = *addr_as_int;
    unsigned int assumed;
    unsigned int value_as_int = __float_as_int(value);

    do {
        assumed = old_int;
        old = __int_as_float(old_int);
        if (value >= old) { // Our value is not better
            break;
        }
        // Try to swap using atomicCAS, comparing integer representations
        old_int = atomicCAS(addr_as_int, assumed, value_as_int);
    } while (assumed != old_int); // Loop if CAS failed

    return old; // Return the value previously stored
}


extern "C" __global__
void dijkstra_relax_kernel(const int* indptr, const int* indices, const float* weights,
                           float* dist, const bool* active_nodes_in, bool* active_nodes_out,
                           bool* changed, int num_nodes) {

    int u = blockIdx.x * blockDim.x + threadIdx.x;

    if (u >= num_nodes) return; // Boundary check first

    bool is_active = active_nodes_in[u];
    active_nodes_out[u] = is_active; // Initialize output state

    if (!is_active) { // Process only active nodes
        return;
    }

    // Node 'u' is active in this iteration. Assume it won't activate others.
    active_nodes_out[u] = false; // Reset output state for 'u'
    float u_dist = dist[u];

    if (u_dist == FLT_MAX) return;

    int start_edge = indptr[u];
    int end_edge = indptr[u + 1];

    for (int i = start_edge; i < end_edge; ++i) {
        int v = indices[i];
        float weight = weights[i];

        if (v < 0 || v >= num_nodes) continue;

        float new_dist_v = u_dist + weight;

        // Use the atomicMinFloat helper function
        float old_dist_v = atomicMinFloat(&dist[v], new_dist_v);

        // If we successfully updated the distance (new_dist_v was smaller than old_dist_v)
        if (new_dist_v < old_dist_v) {
             active_nodes_out[v] = true; // Mark neighbor 'v' as active for the next iteration
             // Indicate that *some* change occurred using atomicCAS on the bool flag (as int)
             atomicCAS((unsigned int*)&changed[0], 0, 1); // Set changed[0] = true (1)
        }
    }
}
'''

# Compile the kernel
dijkstra_module = compiler.SourceModule(dijkstra_kernel_code_pycuda)
dijkstra_relax_kernel_pycuda = dijkstra_module.get_function("dijkstra_relax_kernel")


def run_dijkstra_gpu_timed_pycuda(indptr_cpu, indices_cpu, weights_cpu, num_nodes, start_node=0, max_iterations=1000):
    """ Executes Dijkstra on the GPU using PyCUDA with timing """
    print(f"\nRunning Dijkstra from node {start_node} on GPU (PyCUDA)...")

    infinity = np.finfo(np.float32).max

    # Allocate GPU memory
    d_indptr = drv.mem_alloc(indptr_cpu.nbytes)
    d_indices = drv.mem_alloc(indices_cpu.nbytes)
    d_weights = drv.mem_alloc(weights_cpu.nbytes)
    d_dist = drv.mem_alloc(num_nodes * np.float32().itemsize)
    # PyCUDA bool handling: Often maps to uint8
    bool_itemsize = np.dtype(np.bool_).itemsize
    d_active1 = drv.mem_alloc(num_nodes * bool_itemsize)
    d_active2 = drv.mem_alloc(num_nodes * bool_itemsize)
    d_changed = drv.mem_alloc(1 * bool_itemsize) # Single boolean flag

    # Transfer static data H->D
    drv.memcpy_htod(d_indptr, indptr_cpu)
    drv.memcpy_htod(d_indices, indices_cpu)
    drv.memcpy_htod(d_weights, weights_cpu)

    # Initialize CPU arrays
    dist_cpu = np.full(num_nodes, infinity, dtype=np.float32)
    active1_cpu = np.zeros(num_nodes, dtype=np.bool_)
    active2_cpu = np.zeros(num_nodes, dtype=np.bool_) # Not strictly needed for init
    changed_cpu = np.zeros(1, dtype=np.bool_)

    # Set initial state
    dist_cpu[start_node] = 0.0
    active1_cpu[start_node] = True

    # Transfer initial state H->D
    drv.memcpy_htod(d_dist, dist_cpu)
    drv.memcpy_htod(d_active1, active1_cpu)
    # No need to transfer active2 initially

    active_in_gpu = d_active1
    active_out_gpu = d_active2

    threads_per_block = 256
    grid_dim, block_dim = get_kernel_config_pycuda(num_nodes, threads_per_block)

    # Timing events
    start_event = drv.Event()
    end_event = drv.Event()
    iterations_done = 0

    # --- Main Relaxation Loop ---
    start_event.record()
    for iteration in range(max_iterations):
        iterations_done = iteration + 1

        # Reset changed flag H->D (set to false/0)
        changed_cpu[0] = False
        drv.memcpy_htod(d_changed, changed_cpu)

        # Launch relaxation kernel
        # Note: Pass bool arrays directly. PyCUDA handles pointer passing.
        # Scalars need explicit numpy typing.
        args = [d_indptr, d_indices, d_weights, d_dist,
                active_in_gpu, active_out_gpu, d_changed, np.int32(num_nodes)]
        dijkstra_relax_kernel_pycuda(*args, block=block_dim, grid=grid_dim)

        # Check if any distances were updated D->H
        drv.memcpy_dtoh(changed_cpu, d_changed) # This syncs kernel implicitly
        if not changed_cpu[0]:
            print(f"Convergence reached after {iteration + 1} iterations.")
            break

        # Swap active node pointers
        active_in_gpu, active_out_gpu = active_out_gpu, active_in_gpu

    else: # Loop finished without break
        print(f"Warning: Max iterations ({max_iterations}) reached without full convergence.")

    end_event.record()
    # --- End Relaxation Loop ---

    end_event.synchronize()
    gpu_time = start_event.time_till(end_event) / 1000.0

    # Retrieve results D->H
    dist_result_cpu = np.empty_like(dist_cpu)
    drv.memcpy_dtoh(dist_result_cpu, d_dist)

    # Free GPU memory
    d_indptr.free()
    d_indices.free()
    d_weights.free()
    d_dist.free()
    d_active1.free()
    d_active2.free()
    d_changed.free()

    print(f"GPU Dijkstra (PyCUDA) finished in {gpu_time:.4f} seconds after {iterations_done} iterations.")
    reachable_count = np.sum(dist_result_cpu != infinity)
    print(f"Number of nodes reachable from start node {start_node}: {reachable_count}")
    if iterations_done == max_iterations and changed_cpu[0]:
         print(f"Warning: Max iterations ({max_iterations}) reached without full convergence.")


    return dist_result_cpu, gpu_time


# --- 2.3 PageRank with PyCUDA ---

# Requires a reduction kernel for convergence check
pagerank_kernels_pycuda = r'''
// Includes kernel from CuPy version + reduction kernel

// --- Kernel 1: Calculate Contributions & Dangling Sum ---
extern "C" __global__
void pagerank_kernel(const int* indptr, const int* indices, const float* ranks_in, float* ranks_out,
                      const int* out_degree, float* dangling_sum, // dangling_sum[0] used for atomic update
                      int num_nodes) {

    int u = blockIdx.x * blockDim.x + threadIdx.x; // Thread corresponds to node 'u'
    if (u >= num_nodes) return;

    float rank_u = ranks_in[u];
    int degree_u = out_degree[u];

    if (degree_u > 0) {
        float contribution = rank_u / degree_u;
        int start_edge = indptr[u];
        int end_edge = indptr[u + 1];
        for (int i = start_edge; i < end_edge; ++i) {
            int v = indices[i];
            if (v >= 0 && v < num_nodes) {
                 atomicAdd(&ranks_out[v], contribution); // Add contribution to target node's slot
            }
        }
    } else {
        // Dangling node: add its rank to the global dangling sum
        atomicAdd(&dangling_sum[0], rank_u);
    }
}

// --- Kernel 2: Apply Damping Factor & Dangling Contribution ---
extern "C" __global__
void pagerank_finish_kernel(float* ranks_out, const float* dangling_sum, // Read dangling_sum[0]
                             int num_nodes, float damping_factor, float base_rank) {

    int v = blockIdx.x * blockDim.x + threadIdx.x; // Thread corresponds to node 'v'
    if (v >= num_nodes) return;

    float total_dangling_rank = dangling_sum[0]; // Read accumulated sum
    float dangling_contribution = (num_nodes > 0) ? (total_dangling_rank / num_nodes) : 0.0f;

    // ranks_out[v] currently holds the raw sum of contributions from neighbors (from pagerank_kernel)
    ranks_out[v] = base_rank + damping_factor * (ranks_out[v] + dangling_contribution);
}


// --- Kernel 3: Reduction for Convergence Check ---
// Computes Sum(Abs(arr1 - arr2))
__device__ volatile float reduction_shared_mem[256]; // Max threads per block assumed

extern "C" __global__
void reduce_diff_kernel(const float* arr1, const float* arr2, float* result, int n) {
    // extern __shared__ float sdata[]; // Using static shared mem for fixed size
    volatile float* sdata = reduction_shared_mem; // Point to static shared memory and add volatile

    unsigned int tid = threadIdx.x;
    unsigned int i = blockIdx.x * blockDim.x + threadIdx.x; // Global index
    unsigned int gridSize = gridDim.x * blockDim.x; // Total number of threads in grid

    float my_sum = 0.0f;

    // Each thread computes sum for multiple elements if n > gridSize
    while (i < n) {
        my_sum += fabsf(arr1[i] - arr2[i]);
        i += gridSize;
    }
    sdata[tid] = my_sum; // Store thread's partial sum

    __syncthreads();

    // Reduction in shared memory
    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }

    // Block leader adds block's sum to global result atomically
    if (tid == 0) {
        atomicAdd(result, sdata[0]); // Add block's sum to result[0]
    }
}
'''

# Compile kernels
pagerank_module = compiler.SourceModule(pagerank_kernels_pycuda)
pagerank_kernel_pycuda = pagerank_module.get_function("pagerank_kernel")
pagerank_finish_kernel_pycuda = pagerank_module.get_function("pagerank_finish_kernel")
reduce_diff_kernel_pycuda = pagerank_module.get_function("reduce_diff_kernel")


def run_pagerank_gpu_timed_pycuda(indptr_cpu, indices_cpu, num_nodes, damping_factor=0.85, max_iterations=100, tolerance=1e-5):
    """ Executes PageRank on the GPU using PyCUDA with timing """
    print(f"\nRunning PageRank (d={damping_factor}, tol={tolerance}, max_iter={max_iterations}) on GPU (PyCUDA)...")

    # Precompute Out-Degrees (CPU)
    out_degree_cpu = np.zeros(num_nodes, dtype=np.int32)
    for i in range(num_nodes):
        out_degree_cpu[i] = indptr_cpu[i+1] - indptr_cpu[i]

    # Allocate GPU memory
    d_indptr = drv.mem_alloc(indptr_cpu.nbytes)
    d_indices = drv.mem_alloc(indices_cpu.nbytes)
    d_out_degree = drv.mem_alloc(out_degree_cpu.nbytes)
    d_ranks1 = drv.mem_alloc(num_nodes * np.float32().itemsize) # Ranks In
    d_ranks2 = drv.mem_alloc(num_nodes * np.float32().itemsize) # Ranks Out (buffer)
    d_dangling_sum = drv.mem_alloc(1 * np.float32().itemsize) # Single float for atomic sum
    d_diff_sum = drv.mem_alloc(1 * np.float32().itemsize)     # Single float for reduction result

    # Transfer static data H->D
    drv.memcpy_htod(d_indptr, indptr_cpu)
    drv.memcpy_htod(d_indices, indices_cpu)
    drv.memcpy_htod(d_out_degree, out_degree_cpu)

    # Initialize ranks H->D
    ranks_init_cpu = np.full(num_nodes, 1.0 / num_nodes, dtype=np.float32)
    drv.memcpy_htod(d_ranks1, ranks_init_cpu)

    # Zero buffers for accumulation (use memset for efficiency)
    drv.memset_d32(d_ranks2, 0, num_nodes) # Zero out ranks buffer
    drv.memset_d32(d_dangling_sum, 0, 1)   # Zero out dangling sum
    drv.memset_d32(d_diff_sum, 0, 1)       # Zero out diff sum buffer

    ranks_in_gpu = d_ranks1
    ranks_out_gpu = d_ranks2

    base_rank = np.float32((1.0 - damping_factor) / num_nodes)
    threads_per_block = 256
    grid_dim, block_dim = get_kernel_config_pycuda(num_nodes, threads_per_block)
    # Shared memory for reduction kernel (must be >= threads_per_block * sizeof(float))
    shared_mem_bytes = threads_per_block * np.float32().itemsize

    # Timing events
    start_event = drv.Event()
    end_event = drv.Event()
    iterations_done = 0
    diff_val = infinity = np.finfo(np.float32).max # Initialize diff

    # --- Main PageRank Iteration Loop ---
    start_event.record()
    for iteration in range(max_iterations):
        iterations_done = iteration + 1

        # 1. Reset ranks_out buffer and dangling sum (already zeroed initially, need reset inside loop)
        drv.memset_d32(ranks_out_gpu, 0, num_nodes)
        drv.memset_d32(d_dangling_sum, 0, 1)

        # 2. Launch kernel to calculate contributions and dangling sum
        # Args: indptr, indices, ranks_in, ranks_out, out_degree, dangling_sum_arr, N
        args_k1 = [d_indptr, d_indices, ranks_in_gpu, ranks_out_gpu, d_out_degree,
                   d_dangling_sum, np.int32(num_nodes)]
        pagerank_kernel_pycuda(*args_k1, block=block_dim, grid=grid_dim)

        # 3. Launch kernel to apply damping factor, base rank, and dangling contribution
        # Args: ranks_out, dangling_sum_arr, N, d, base_rank
        args_k2 = [ranks_out_gpu, d_dangling_sum, np.int32(num_nodes),
                   np.float32(damping_factor), base_rank] # Pass base_rank correctly
        pagerank_finish_kernel_pycuda(*args_k2, block=block_dim, grid=grid_dim)

        # 4. Check for convergence using reduction kernel
        drv.memset_d32(d_diff_sum, 0, 1) # Reset diff sum buffer before reduction
        # Args: ranks_out, ranks_in, result_buffer, N
        args_k3 = [ranks_out_gpu, ranks_in_gpu, d_diff_sum, np.int32(num_nodes)]
        reduce_diff_kernel_pycuda(*args_k3, block=block_dim, grid=grid_dim, shared=shared_mem_bytes)

        # Retrieve the difference sum D->H
        diff_sum_h = np.zeros(1, dtype=np.float32)
        drv.memcpy_dtoh(diff_sum_h, d_diff_sum) # This syncs reduction kernel implicitly
        diff_val = diff_sum_h[0]

        if iteration % 10 == 0 or iteration == max_iterations - 1 or diff_val < tolerance:
             print(f"  Iteration {iteration+1}: Change = {diff_val:.6f}")

        if diff_val < tolerance:
            print(f"Convergence reached after {iteration + 1} iterations.")
            # ranks_out_gpu contains the final converged ranks, update pointer
            ranks_in_gpu = ranks_out_gpu
            break

        # 5. Swap rank pointers (no data movement)
        ranks_in_gpu, ranks_out_gpu = ranks_out_gpu, ranks_in_gpu
    else: # Loop finished without break
        # ranks_in_gpu holds the result from the last completed iteration
        print(f"Warning: Max iterations ({max_iterations}) reached without convergence.")


    end_event.record()
    # --- End PageRank Loop ---

    end_event.synchronize()
    gpu_time = start_event.time_till(end_event) / 1000.0

    # Retrieve final ranks D->H (pointed to by ranks_in_gpu)
    ranks_result_cpu = np.empty_like(ranks_init_cpu)
    drv.memcpy_dtoh(ranks_result_cpu, ranks_in_gpu)

    # Free GPU memory
    d_indptr.free()
    d_indices.free()
    d_out_degree.free()
    d_ranks1.free()
    d_ranks2.free()
    d_dangling_sum.free()
    d_diff_sum.free()

    print(f"GPU PageRank (PyCUDA) finished in {gpu_time:.4f} seconds after {iterations_done} iterations.")
    print(f"Sum of ranks: {np.sum(ranks_result_cpu):.4f}")
    if iterations_done == max_iterations and diff_val >= tolerance:
        print(f"Warning: Max iterations ({max_iterations}) reached without convergence (last change: {diff_val:.6f}).")

    return ranks_result_cpu, gpu_time



/usr/local/lib/python3.11/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()


In [ ]:
# @title 2. PyCUDA Kernels and Host Code (Corrected PageRank Kernels)


# --- BFS Code (No changes needed) ---
bfs_kernel_code_pycuda = r'''
extern "C" __global__
void bfs_kernel(const int* indptr, const int* indices, int* dist, int* frontier_in, int* frontier_out, int* frontier_size_out, int current_level, int num_nodes) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < frontier_size_out[0]) {
         int u = frontier_in[tid];
         if (u < 0 || u >= num_nodes) return;
         int start_edge = indptr[u];
         int end_edge = indptr[u + 1];
         for (int i = start_edge; i < end_edge; ++i) {
             int v = indices[i];
             if (v < 0 || v >= num_nodes) continue;
             int old_dist = atomicCAS(&dist[v], -1, current_level + 1);
             if (old_dist == -1) {
                 int index = atomicAdd(&frontier_size_out[1], 1);
                 frontier_out[index] = v;
             }
         }
    }
}
'''
bfs_module = compiler.SourceModule(bfs_kernel_code_pycuda)
bfs_kernel_pycuda = bfs_module.get_function("bfs_kernel")
# --- End BFS Code ---


# --- Dijkstra Code (No changes needed) ---
dijkstra_kernel_code_pycuda = r'''
#include <float.h>

static __device__ float atomicMinFloat(float* addr, float value) {
    float old;
    unsigned int* addr_as_int = (unsigned int*)addr;
    unsigned int old_int = *addr_as_int;
    unsigned int assumed;
    unsigned int value_as_int = __float_as_int(value);
    do {
        assumed = old_int;
        old = __int_as_float(old_int);
        if (value >= old) { break; }
        old_int = atomicCAS(addr_as_int, assumed, value_as_int);
    } while (assumed != old_int);
    return old;
}

extern "C" __global__
void dijkstra_relax_kernel(const int* indptr, const int* indices, const float* weights,
                           float* dist, const bool* active_nodes_in, bool* active_nodes_out,
                           bool* changed, int num_nodes) {
    int u = blockIdx.x * blockDim.x + threadIdx.x;
    if (u >= num_nodes) return;
    bool is_active = active_nodes_in[u];
    active_nodes_out[u] = is_active;
    if (!is_active) { return; }
    active_nodes_out[u] = false;
    float u_dist = dist[u];
    if (u_dist == FLT_MAX) return;
    int start_edge = indptr[u];
    int end_edge = indptr[u + 1];
    for (int i = start_edge; i < end_edge; ++i) {
        int v = indices[i];
        float weight = weights[i];
        if (v < 0 || v >= num_nodes) continue;
        float new_dist_v = u_dist + weight;
        float old_dist_v = atomicMinFloat(&dist[v], new_dist_v);
        if (new_dist_v < old_dist_v) {
             active_nodes_out[v] = true;
             atomicCAS((unsigned int*)&changed[0], 0, 1);
        }
    }
}
'''
dijkstra_module = compiler.SourceModule(dijkstra_kernel_code_pycuda)
dijkstra_relax_kernel_pycuda = dijkstra_module.get_function("dijkstra_relax_kernel")
# --- End Dijkstra Code ---


# --- PageRank Kernels (CORRECTED) ---
pagerank_kernels_pycuda_corrected = r'''
// Kernel 1: Calculate Contributions & Dangling Sum (No change)
extern "C" __global__
void pagerank_kernel(const int* indptr, const int* indices, const float* ranks_in, float* ranks_out,
                      const int* out_degree, float* dangling_sum,
                      int num_nodes) {
    int u = blockIdx.x * blockDim.x + threadIdx.x;
    if (u >= num_nodes) return;
    float rank_u = ranks_in[u];
    int degree_u = out_degree[u];
    if (degree_u > 0) {
        float contribution = rank_u / degree_u;
        int start_edge = indptr[u];
        int end_edge = indptr[u + 1];
        for (int i = start_edge; i < end_edge; ++i) {
            int v = indices[i];
            if (v >= 0 && v < num_nodes) {
                 atomicAdd(&ranks_out[v], contribution);
            }
        }
    } else {
        atomicAdd(&dangling_sum[0], rank_u);
    }
}

// Kernel 2: Apply Damping Factor & Dangling Contribution (No change)
extern "C" __global__
void pagerank_finish_kernel(float* ranks_out, const float* dangling_sum,
                             int num_nodes, float damping_factor, float base_rank) {
    int v = blockIdx.x * blockDim.x + threadIdx.x;
    if (v >= num_nodes) return;
    float total_dangling_rank = dangling_sum[0];
    float dangling_contribution = (num_nodes > 0) ? (total_dangling_rank / num_nodes) : 0.0f;
    ranks_out[v] = base_rank + damping_factor * (ranks_out[v] + dangling_contribution);
}


// Kernel 3: Reduction for Convergence Check (CORRECTED)
extern "C" __global__
void reduce_diff_kernel(const float* arr1, const float* arr2, float* result, int n) {
    // Use dynamically sized shared memory, declared via 'shared=' parameter in kernel launch
    extern __shared__ float sdata[];

    unsigned int tid = threadIdx.x;
    unsigned int i = blockIdx.x * blockDim.x + threadIdx.x; // Global index
    unsigned int gridSize = gridDim.x * blockDim.x; // Total number of threads in grid

    float my_sum = 0.0f;

    // Each thread computes sum for multiple elements if n > gridSize (grid-stride loop)
    while (i < n) {
        my_sum += fabsf(arr1[i] - arr2[i]);
        i += gridSize;
    }
    sdata[tid] = my_sum; // Store thread's partial sum into shared memory

    __syncthreads(); // Wait for all threads in block to finish calculating and storing

    // Perform reduction in shared memory
    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads(); // Wait for partial sum to complete at each step
    }

    // Block leader (thread 0) adds this block's total sum to the global result atomically
    if (tid == 0) {
        atomicAdd(result, sdata[0]);
    }
}
'''

# Compile the corrected PageRank kernels
try:
    pagerank_module = compiler.SourceModule(pagerank_kernels_pycuda_corrected)
    pagerank_kernel_pycuda = pagerank_module.get_function("pagerank_kernel")
    pagerank_finish_kernel_pycuda = pagerank_module.get_function("pagerank_finish_kernel")
    reduce_diff_kernel_pycuda = pagerank_module.get_function("reduce_diff_kernel")
    print("PageRank kernels compiled successfully.")
except compiler.CompileError as e:
    print("ERROR compiling PageRank kernels:")
    print(e)
    # Handle error appropriately, maybe exit or skip PageRank


# --- Host Code (run_..._gpu_timed_pycuda functions) ---
# Make sure the Python functions run_bfs_gpu_timed_pycuda, run_dijkstra_gpu_timed_pycuda,
# and run_pagerank_gpu_timed_pycuda defined in the previous PyCUDA block are present.
# The run_pagerank_gpu_timed_pycuda function already correctly calculates
# shared_mem_bytes and passes it via the 'shared=' argument when launching
# reduce_diff_kernel_pycuda, so no changes are needed in the host code for that part.

# --- Re-paste host code if needed ---
def run_bfs_gpu_timed_pycuda(indptr_cpu, indices_cpu, num_nodes, start_node=0):
    print(f"\nRunning BFS from node {start_node} on GPU (PyCUDA)...")
    d_indptr = drv.mem_alloc(indptr_cpu.nbytes)
    d_indices = drv.mem_alloc(indices_cpu.nbytes)
    d_dist = drv.mem_alloc(num_nodes * np.int32().itemsize)
    d_frontier1 = drv.mem_alloc(num_nodes * np.int32().itemsize)
    d_frontier2 = drv.mem_alloc(num_nodes * np.int32().itemsize)
    d_frontier_size = drv.mem_alloc(2 * np.int32().itemsize)
    drv.memcpy_htod(d_indptr, indptr_cpu)
    drv.memcpy_htod(d_indices, indices_cpu)
    dist_init_cpu = np.full(num_nodes, -1, dtype=np.int32)
    drv.memcpy_htod(d_dist, dist_init_cpu)
    frontier_size_init_cpu = np.zeros(2, dtype=np.int32)
    drv.memcpy_htod(d_frontier_size, frontier_size_init_cpu)
    dist_init_cpu[start_node] = 0
    drv.memcpy_htod(d_dist, dist_init_cpu)
    frontier1_init_cpu = np.zeros(num_nodes, dtype=np.int32)
    frontier1_init_cpu[0] = start_node
    drv.memcpy_htod(d_frontier1, frontier1_init_cpu)
    frontier_size_init_cpu[0] = 1
    drv.memcpy_htod(d_frontier_size, frontier_size_init_cpu)
    current_level = 0
    threads_per_block = 256
    frontier_in_gpu = d_frontier1
    frontier_out_gpu = d_frontier2
    start_event = drv.Event()
    end_event = drv.Event()
    frontier_size_h = np.zeros(2, dtype=np.int32)
    frontier_size_h_reset_next = np.zeros(2, dtype=np.int32)
    start_event.record()
    while True:
        drv.memcpy_dtoh(frontier_size_h, d_frontier_size)
        current_frontier_size = int(frontier_size_h[0])
        if current_frontier_size == 0: break
        frontier_size_h_reset_next[0] = current_frontier_size
        frontier_size_h_reset_next[1] = 0
        drv.memcpy_htod(d_frontier_size, frontier_size_h_reset_next)
        grid_dim, block_dim = get_kernel_config_pycuda(current_frontier_size, threads_per_block)
        args = [d_indptr, d_indices, d_dist, frontier_in_gpu, frontier_out_gpu,
                d_frontier_size, np.int32(current_level), np.int32(num_nodes)]
        bfs_kernel_pycuda(*args, block=block_dim, grid=grid_dim)
        drv.memcpy_dtoh(frontier_size_h, d_frontier_size)
        next_frontier_size = int(frontier_size_h[1])
        frontier_size_h[0] = next_frontier_size
        drv.memcpy_htod(d_frontier_size, frontier_size_h)
        frontier_in_gpu, frontier_out_gpu = frontier_out_gpu, frontier_in_gpu
        current_level += 1
    end_event.record()
    end_event.synchronize()
    gpu_time = start_event.time_till(end_event) / 1000.0
    dist_result_cpu = np.empty_like(dist_init_cpu)
    drv.memcpy_dtoh(dist_result_cpu, d_dist)
    d_indptr.free(); d_indices.free(); d_dist.free(); d_frontier1.free(); d_frontier2.free(); d_frontier_size.free()
    print(f"GPU BFS (PyCUDA) finished in {gpu_time:.4f} seconds.")
    unvisited_count = np.sum(dist_result_cpu == -1)
    print(f"Number of nodes reachable from start node {start_node}: {num_nodes - unvisited_count}")
    return dist_result_cpu, gpu_time

def run_dijkstra_gpu_timed_pycuda(indptr_cpu, indices_cpu, weights_cpu, num_nodes, start_node=0, max_iterations=1000):
    print(f"\nRunning Dijkstra from node {start_node} on GPU (PyCUDA)...")
    infinity = np.finfo(np.float32).max
    d_indptr = drv.mem_alloc(indptr_cpu.nbytes); drv.memcpy_htod(d_indptr, indptr_cpu)
    d_indices = drv.mem_alloc(indices_cpu.nbytes); drv.memcpy_htod(d_indices, indices_cpu)
    d_weights = drv.mem_alloc(weights_cpu.nbytes); drv.memcpy_htod(d_weights, weights_cpu)
    d_dist = drv.mem_alloc(num_nodes * np.float32().itemsize)
    bool_itemsize = np.dtype(np.bool_).itemsize
    d_active1 = drv.mem_alloc(num_nodes * bool_itemsize)
    d_active2 = drv.mem_alloc(num_nodes * bool_itemsize)
    d_changed = drv.mem_alloc(1 * bool_itemsize)
    dist_cpu = np.full(num_nodes, infinity, dtype=np.float32)
    active1_cpu = np.zeros(num_nodes, dtype=np.bool_)
    changed_cpu = np.zeros(1, dtype=np.bool_)
    dist_cpu[start_node] = 0.0; active1_cpu[start_node] = True
    drv.memcpy_htod(d_dist, dist_cpu)
    drv.memcpy_htod(d_active1, active1_cpu)
    active_in_gpu = d_active1; active_out_gpu = d_active2
    threads_per_block = 256
    grid_dim, block_dim = get_kernel_config_pycuda(num_nodes, threads_per_block)
    start_event = drv.Event(); end_event = drv.Event()
    iterations_done = 0
    start_event.record()
    for iteration in range(max_iterations):
        iterations_done = iteration + 1
        changed_cpu[0] = False; drv.memcpy_htod(d_changed, changed_cpu)
        args = [d_indptr, d_indices, d_weights, d_dist,
                active_in_gpu, active_out_gpu, d_changed, np.int32(num_nodes)]
        dijkstra_relax_kernel_pycuda(*args, block=block_dim, grid=grid_dim)
        drv.memcpy_dtoh(changed_cpu, d_changed)
        if not changed_cpu[0]: print(f"Convergence reached after {iteration + 1} iterations."); break
        active_in_gpu, active_out_gpu = active_out_gpu, active_in_gpu
    else: print(f"Warning: Max iterations ({max_iterations}) reached without full convergence.")
    end_event.record()
    end_event.synchronize()
    gpu_time = start_event.time_till(end_event) / 1000.0
    dist_result_cpu = np.empty_like(dist_cpu)
    drv.memcpy_dtoh(dist_result_cpu, d_dist)
    d_indptr.free(); d_indices.free(); d_weights.free(); d_dist.free(); d_active1.free(); d_active2.free(); d_changed.free()
    print(f"GPU Dijkstra (PyCUDA) finished in {gpu_time:.4f} seconds after {iterations_done} iterations.")
    reachable_count = np.sum(dist_result_cpu != infinity)
    print(f"Number of nodes reachable from start node {start_node}: {reachable_count}")
    if iterations_done == max_iterations and changed_cpu[0]: print(f"Warning: Max iterations ({max_iterations}) reached without full convergence.")
    return dist_result_cpu, gpu_time


def run_pagerank_gpu_timed_pycuda(indptr_cpu, indices_cpu, num_nodes, damping_factor=0.85, max_iterations=100, tolerance=1e-5):
    print(f"\nRunning PageRank (d={damping_factor}, tol={tolerance}, max_iter={max_iterations}) on GPU (PyCUDA)...")
    out_degree_cpu = np.zeros(num_nodes, dtype=np.int32)
    for i in range(num_nodes): out_degree_cpu[i] = indptr_cpu[i+1] - indptr_cpu[i]
    d_indptr = drv.mem_alloc(indptr_cpu.nbytes); drv.memcpy_htod(d_indptr, indptr_cpu)
    d_indices = drv.mem_alloc(indices_cpu.nbytes); drv.memcpy_htod(d_indices, indices_cpu)
    d_out_degree = drv.mem_alloc(out_degree_cpu.nbytes); drv.memcpy_htod(d_out_degree, out_degree_cpu)
    d_ranks1 = drv.mem_alloc(num_nodes * np.float32().itemsize)
    d_ranks2 = drv.mem_alloc(num_nodes * np.float32().itemsize)
    d_dangling_sum = drv.mem_alloc(1 * np.float32().itemsize)
    d_diff_sum = drv.mem_alloc(1 * np.float32().itemsize)
    ranks_init_cpu = np.full(num_nodes, 1.0 / num_nodes, dtype=np.float32)
    drv.memcpy_htod(d_ranks1, ranks_init_cpu)
    drv.memset_d32(d_ranks2, 0, num_nodes)
    drv.memset_d32(d_dangling_sum, 0, 1)
    drv.memset_d32(d_diff_sum, 0, 1)
    ranks_in_gpu = d_ranks1; ranks_out_gpu = d_ranks2
    base_rank = np.float32((1.0 - damping_factor) / num_nodes)
    threads_per_block = 256
    grid_dim, block_dim = get_kernel_config_pycuda(num_nodes, threads_per_block)
    shared_mem_bytes = threads_per_block * np.float32().itemsize
    start_event = drv.Event(); end_event = drv.Event()
    iterations_done = 0
    diff_val = np.finfo(np.float32).max
    start_event.record()
    for iteration in range(max_iterations):
        iterations_done = iteration + 1
        drv.memset_d32(ranks_out_gpu, 0, num_nodes)
        drv.memset_d32(d_dangling_sum, 0, 1)
        args_k1 = [d_indptr, d_indices, ranks_in_gpu, ranks_out_gpu, d_out_degree, d_dangling_sum, np.int32(num_nodes)]
        pagerank_kernel_pycuda(*args_k1, block=block_dim, grid=grid_dim)
        args_k2 = [ranks_out_gpu, d_dangling_sum, np.int32(num_nodes), np.float32(damping_factor), base_rank]
        pagerank_finish_kernel_pycuda(*args_k2, block=block_dim, grid=grid_dim)
        drv.memset_d32(d_diff_sum, 0, 1)
        args_k3 = [ranks_out_gpu, ranks_in_gpu, d_diff_sum, np.int32(num_nodes)]
        reduce_diff_kernel_pycuda(*args_k3, block=block_dim, grid=grid_dim, shared=shared_mem_bytes)
        diff_sum_h = np.zeros(1, dtype=np.float32)
        drv.memcpy_dtoh(diff_sum_h, d_diff_sum)
        diff_val = diff_sum_h[0]
        #if iteration % 10 == 0 or iteration == max_iterations - 1 or diff_val < tolerance: print(f"  Iteration {iteration+1}: Change = {diff_val:.6f}")
        if diff_val < tolerance: print(f"Convergence reached after {iteration + 1} iterations."); ranks_in_gpu = ranks_out_gpu; break
        ranks_in_gpu, ranks_out_gpu = ranks_out_gpu, ranks_in_gpu
    else: print(f"Warning: Max iterations ({max_iterations}) reached without convergence.")
    end_event.record()
    end_event.synchronize()
    gpu_time = start_event.time_till(end_event) / 1000.0
    ranks_result_cpu = np.empty_like(ranks_init_cpu)
    drv.memcpy_dtoh(ranks_result_cpu, ranks_in_gpu)
    d_indptr.free(); d_indices.free(); d_out_degree.free(); d_ranks1.free(); d_ranks2.free(); d_dangling_sum.free(); d_diff_sum.free()
    print(f"GPU PageRank (PyCUDA) finished in {gpu_time:.4f} seconds after {iterations_done} iterations.")
    print(f"Sum of ranks: {np.sum(ranks_result_cpu):.4f}")
    if iterations_done == max_iterations and diff_val >= tolerance: print(f"Warning: Max iterations ({max_iterations}) reached without convergence (last change: {diff_val:.6f}).")
    return ranks_result_cpu, gpu_time

# --- End Host Code ---

/usr/local/lib/python3.11/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()
/usr/local/lib/python3.11/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()


PageRank kernels compiled successfully.


In [ ]:
# @title 3. CPU Implementations



import collections
import heapq

def run_bfs_cpu(indptr, indices, num_nodes, start_node=0):
    print(f"\nRunning BFS from node {start_node} on CPU...")
    start_time = time.time()
    dist = np.full(num_nodes, -1, dtype=np.int32)
    if start_node < 0 or start_node >= num_nodes: return dist
    dist[start_node] = 0
    queue = collections.deque([start_node])
    visited_count = 1
    while queue:
        u = queue.popleft()
        current_level = dist[u]
        start_edge, end_edge = indptr[u], indptr[u + 1]
        for i in range(start_edge, end_edge):
            v = indices[i]
            if v >= 0 and v < num_nodes and dist[v] == -1:
                dist[v] = current_level + 1
                queue.append(v)
                visited_count +=1
    end_time = time.time()
    print(f"CPU BFS finished in {end_time - start_time:.4f} seconds.")
    print(f"Number of nodes reachable from start node {start_node}: {visited_count}")
    return dist

def run_dijkstra_cpu(indptr, indices, weights, num_nodes, start_node=0):
    print(f"\nRunning Dijkstra from node {start_node} on CPU...")
    start_time = time.time()
    infinity = np.finfo(np.float32).max
    dist = np.full(num_nodes, infinity, dtype=np.float32)
    visited = np.zeros(num_nodes, dtype=bool)
    if start_node < 0 or start_node >= num_nodes: return dist
    dist[start_node] = 0.0
    pq = [(0.0, start_node)]
    nodes_processed = 0
    while pq:
        d_u, u = heapq.heappop(pq)
        if d_u > dist[u] or visited[u]: continue
        visited[u] = True
        nodes_processed += 1
        start_edge, end_edge = indptr[u], indptr[u + 1]
        for i in range(start_edge, end_edge):
            v = indices[i]
            weight = weights[i]
            if v >= 0 and v < num_nodes and not visited[v]:
                new_dist_v = dist[u] + weight
                if new_dist_v < dist[v]:
                    dist[v] = new_dist_v
                    heapq.heappush(pq, (new_dist_v, v))
    end_time = time.time()
    print(f"CPU Dijkstra finished in {end_time - start_time:.4f} seconds.")
    reachable_count = np.sum(dist != infinity)
    print(f"Number of nodes reachable from start node {start_node}: {reachable_count}")
    return dist

def run_pagerank_cpu(indptr, indices, num_nodes, damping_factor=0.85, max_iterations=100, tolerance=1e-5):
    print(f"\nRunning PageRank (d={damping_factor}, tol={tolerance}, max_iter={max_iterations}) on CPU...")
    start_time = time.time()
    out_degree = np.zeros(num_nodes, dtype=np.int32)
    for i in range(num_nodes): out_degree[i] = indptr[i+1] - indptr[i]
    dangling_nodes = np.where(out_degree == 0)[0]
    ranks = np.full(num_nodes, 1.0 / num_nodes, dtype=np.float32)
    next_ranks = np.empty(num_nodes, dtype=np.float32)
    base_rank = (1.0 - damping_factor) / num_nodes
    for iteration in range(max_iterations):
        next_ranks.fill(0.0)
        dangling_sum = np.sum(ranks[dangling_nodes])
        dangling_contribution = dangling_sum / num_nodes
        for u in range(num_nodes):
            if out_degree[u] > 0:
                contribution = ranks[u] / out_degree[u]
                start_edge, end_edge = indptr[u], indptr[u+1]
                for i in range(start_edge, end_edge):
                    v = indices[i]
                    if v >= 0 and v < num_nodes: next_ranks[v] += contribution
        next_ranks = base_rank + damping_factor * (next_ranks + dangling_contribution)
        diff = np.sum(np.abs(next_ranks - ranks))
        ranks, next_ranks = next_ranks, ranks
        # if iteration % 10 == 0 or iteration == max_iterations - 1: print(f"  Iteration {iteration+1}: Change = {diff:.6f}") # Less verbose
        if diff < tolerance:
            print(f"Convergence reached after {iteration + 1} iterations.")
            break
    else: print(f"Warning: Max iterations ({max_iterations}) reached without convergence.")
    end_time = time.time()
    print(f"CPU PageRank finished in {end_time - start_time:.4f} seconds.")
    print(f"Sum of ranks: {np.sum(ranks):.4f}")
    return ranks

In [ ]:
# @title 4. Comparison: CPU vs GPU (PyCUDA)

print("\n--- Performance Comparison (PyCUDA) ---")

start_node = 0 # Use the same start node

# --- BFS Comparison ---
print("\n--- BFS ---")
cpu_bfs_start_time = time.time()
bfs_dist_cpu = run_bfs_cpu(csr_indptr_cpu, csr_indices_cpu, N, start_node=start_node)
cpu_bfs_end_time = time.time()
cpu_bfs_time = cpu_bfs_end_time - cpu_bfs_start_time

# Run PyCUDA BFS
bfs_dist_gpu, gpu_bfs_time_accurate = run_bfs_gpu_timed_pycuda(csr_indptr_cpu, csr_indices_cpu, N, start_node=start_node)

print("\nChecking BFS Correctness...")
if np.array_equal(bfs_dist_cpu, bfs_dist_gpu):
    print("BFS CPU and GPU (PyCUDA) results MATCH.")
else:
    print("ERROR: BFS CPU and GPU (PyCUDA) results DO NOT MATCH.")

print(f"BFS CPU Time: {cpu_bfs_time:.4f} s")
print(f"BFS GPU Time (PyCUDA): {gpu_bfs_time_accurate:.4f} s")
if gpu_bfs_time_accurate > 0:
    print(f"BFS Speedup (CPU/GPU): {cpu_bfs_time / gpu_bfs_time_accurate:.2f}x")
else: print("BFS Speedup: GPU time was zero or negative.")


# --- Dijkstra Comparison ---
print("\n--- Dijkstra ---")
cpu_dijkstra_start_time = time.time()
dijkstra_dist_cpu = run_dijkstra_cpu(csr_indptr_cpu, csr_indices_cpu, csr_weights_cpu, N, start_node=start_node)
cpu_dijkstra_end_time = time.time()
cpu_dijkstra_time = cpu_dijkstra_end_time - cpu_dijkstra_start_time

# Run PyCUDA Dijkstra
dijkstra_dist_gpu, gpu_dijkstra_time_accurate = run_dijkstra_gpu_timed_pycuda(csr_indptr_cpu, csr_indices_cpu, csr_weights_cpu, N, start_node=start_node)

print("\nChecking Dijkstra Correctness...")
if np.allclose(dijkstra_dist_cpu, dijkstra_dist_gpu, atol=1e-5, rtol=1e-5):
    print("Dijkstra CPU and GPU (PyCUDA) results MATCH (within tolerance).")
else:
    print("ERROR: Dijkstra CPU and GPU (PyCUDA) results DO NOT MATCH.")

print(f"Dijkstra CPU Time: {cpu_dijkstra_time:.4f} s")
print(f"Dijkstra GPU Time (PyCUDA): {gpu_dijkstra_time_accurate:.4f} s")
if gpu_dijkstra_time_accurate > 0:
    print(f"Dijkstra Speedup (CPU/GPU): {cpu_dijkstra_time / gpu_dijkstra_time_accurate:.2f}x")
else: print("Dijkstra Speedup: GPU time was zero or negative.")


# --- PageRank Comparison ---
print("\n--- PageRank ---")
cpu_pagerank_start_time = time.time()
pagerank_ranks_cpu = run_pagerank_cpu(csr_indptr_cpu, csr_indices_cpu, N)
cpu_pagerank_end_time = time.time()
cpu_pagerank_time = cpu_pagerank_end_time - cpu_pagerank_start_time

# Run PyCUDA PageRank
pagerank_ranks_gpu, gpu_pagerank_time_accurate = run_pagerank_gpu_timed_pycuda(csr_indptr_cpu, csr_indices_cpu, N)

print("\nChecking PageRank Correctness...")
if np.allclose(pagerank_ranks_cpu, pagerank_ranks_gpu, atol=1e-5, rtol=1e-5):
    print("PageRank CPU and GPU (PyCUDA) results MATCH (within tolerance).")
else:
    print("ERROR: PageRank CPU and GPU (PyCUDA) results DO NOT MATCH.")


print(f"PageRank CPU Time: {cpu_pagerank_time:.4f} s")
print(f"PageRank GPU Time (PyCUDA): {gpu_pagerank_time_accurate:.4f} s")
if gpu_pagerank_time_accurate > 0:
    print(f"PageRank Speedup (CPU/GPU): {cpu_pagerank_time / gpu_pagerank_time_accurate:.2f}x")
else: print("PageRank Speedup: GPU time was zero or negative.")


print("\n--- Comparison Summary (PyCUDA) ---")
print(f"Graph Size: {N} nodes, ~{csr_indices_cpu.shape[0]} edges")
print(f"Algorithm | CPU Time (s) | GPU Time (s) | Speedup (CPU/GPU)")
print(f"----------|--------------|--------------|--------------------")
if gpu_bfs_time_accurate > 0: print(f"BFS       | {cpu_bfs_time:>12.4f} | {gpu_bfs_time_accurate:>12.4f} | {cpu_bfs_time / gpu_bfs_time_accurate:>18.2f}x")
else: print(f"BFS       | {cpu_bfs_time:>12.4f} | {gpu_bfs_time_accurate:>12.4f} | {'N/A':>18}")
if gpu_dijkstra_time_accurate > 0: print(f"Dijkstra  | {cpu_dijkstra_time:>12.4f} | {gpu_dijkstra_time_accurate:>12.4f} | {cpu_dijkstra_time / gpu_dijkstra_time_accurate:>18.2f}x")
else: print(f"Dijkstra  | {cpu_dijkstra_time:>12.4f} | {gpu_dijkstra_time_accurate:>12.4f} | {'N/A':>18}")
if gpu_pagerank_time_accurate > 0: print(f"PageRank  | {cpu_pagerank_time:>12.4f} | {gpu_pagerank_time_accurate:>12.4f} | {cpu_pagerank_time / gpu_pagerank_time_accurate:>18.2f}x")
else: print(f"PageRank  | {cpu_pagerank_time:>12.4f} | {gpu_pagerank_time_accurate:>12.4f} | {'N/A':>18}")


--- Performance Comparison (PyCUDA) ---

--- BFS ---

Running BFS from node 0 on CPU...
CPU BFS finished in 0.0533 seconds.
Number of nodes reachable from start node 0: 4096

Running BFS from node 0 on GPU (PyCUDA)...
GPU BFS (PyCUDA) finished in 0.0013 seconds.
Number of nodes reachable from start node 0: 4096

Checking BFS Correctness...
BFS CPU and GPU (PyCUDA) results MATCH.
BFS CPU Time: 0.0535 s
BFS GPU Time (PyCUDA): 0.0013 s
BFS Speedup (CPU/GPU): 41.79x

--- Dijkstra ---

Running Dijkstra from node 0 on CPU...
CPU Dijkstra finished in 0.0884 seconds.
Number of nodes reachable from start node 0: 4096

Running Dijkstra from node 0 on GPU (PyCUDA)...
Convergence reached after 8 iterations.
GPU Dijkstra (PyCUDA) finished in 0.0007 seconds after 8 iterations.
Number of nodes reachable from start node 0: 4096

Checking Dijkstra Correctness...
Dijkstra CPU and GPU (PyCUDA) results MATCH (within tolerance).
Dijkstra CPU Time: 0.0886 s
Dijkstra GPU Time (PyCUDA): 0.0007 s
Dijkstra Spe